Mounted at /content/drive


In [2]:
!pip install sentence_transformers faiss-cpu -q

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from tqdm.autonotebook import tqdm
import numpy as np
import faiss

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
train_data_cat = pd.read_csv("/home/jovyan/lost+found/ma_folder/train_data_categories.csv")
train_data_cat.head(5)

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира


In [5]:
IAB_tags = pd.read_csv("/home/jovyan/lost+found/ma_folder/IAB_tags.csv")
IAB_tags.head()

,Уровень 1 (iab),Уровень 2 (iab),Уровень 3 (iab)
0,Транспорт,NaN,NaN
1,Транспорт,Типы кузова автомобиля,NaN
2,Транспорт,Типы кузова автомобиля,Грузовой автомобиль
3,Транспорт,Типы кузова автомобиля,Седан
4,Транспорт,Типы кузова автомобиля,Универсал


In [6]:
data = train_data_cat
taxonomy = IAB_tags

#Baseline

In [7]:
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence', )
model = model.to('cuda')
dim = 768 # размер вектора эмбеддинга

No sentence-transformers model found with name DeepPavlov/rubert-base-cased-sentence. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
data['title_vector'] = data['title'].apply(lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())

In [ ]:
def get_tags():
    tags = {}
    for i, row in tqdm(taxonomy.iterrows()):
        if isinstance(row['Уровень 1 (iab)'], str):
            tags[row['Уровень 1 (iab)']] = model.encode(row['Уровень 1 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 2 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 3 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
    return tags

tags = get_tags()
tags_list = list(tags.keys())
vectors = np.array(list(tags.values()))

0it [00:00, ?it/s]

In [ ]:
index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
print(index.ntotal)
index.add(vectors)
print(index.ntotal)

0
610


In [ ]:
topn = 3
scores, predictions = index.search(np.array(data['title_vector'].to_list()[:10]), topn)
for j, i in enumerate(predictions):
    print("SCORES", scores[j])
    print("PREDICTION_by_title", np.array(tags_list)[predictions[j]])
    print("SAMPLE", data['title'].to_list()[:10][j])
    print("\n")

SCORES [280.28256 268.94186 264.77997]
PREDICTION_by_title ['Семья и отношения: Развод' 'Массовая культура: Смерти знаменитостей'
 'Массовая культура: Скандалы знаменитостей']
SAMPLE Пацанский клининг. Шоу «ЧистоТачка» | Повелитель ночи | Выпуск 17


SCORES [248.28539 242.65976 241.96158]
PREDICTION_by_title ['Массовая культура: Скандалы знаменитостей' 'Игры: Киберспорт'
 'Спорт: Дартс']
SAMPLE СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Жека Секси


SCORES [291.92664 287.78052 280.69467]
PREDICTION_by_title ['Семья и отношения: Развод'
 'Медицина: Медицинские направления: Простуда и грипп'
 'Новости и политика: Политика: Война и конфликты']
SAMPLE Злые языки | Выпуск 1, Сезон 1 | Непорочность Даны Борисовой


SCORES [247.24626 243.45216 236.21658]
PREDICTION_by_title ['Музыка и аудио: Мировые хиты'
 'События и достопримечательности: Личные события: День рождения'
 'Массовая культура: Скандалы знаменитостей']
SAMPLE $1000 шоу | 1 выпуск | Автобоулинг


SCORES [276.93286 271.74152 269.

In [ ]:
topn=1
sample_submission = pd.DataFrame(data=data['video_id'].to_list(), columns=['video_id'])
sample_submission['predicted_tags']=np.nan
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype('object')

for i, row in data.iterrows():
    scores, predictions = index.search(np.array([row['title_vector']]), topn)
    index_i = sample_submission[sample_submission.video_id == row.video_id].index
    sample_submission.at[index_i[0], 'predicted_tags'] = [tags_list[predictions[0][0]]] # вытаскиваем предсказание из

In [ ]:
type(sample_submission['predicted_tags'].to_list()[0])

list

In [ ]:
data.tags.head()

,tags
0,Массовая культура: Юмор и сатира
1,События и достопримечательности: Спортивные с...
2,"Массовая культура: Отношения знаменитостей, Ма..."
3,"Транспорт, Спорт: Автогонки, Массовая культура"
4,Массовая культура: Юмор и сатира


In [ ]:
sample_submission.predicted_tags.head()

,predicted_tags
0,[Семья и отношения: Развод]
1,[Массовая культура: Скандалы знаменитостей]
2,[Семья и отношения: Развод]
3,[Музыка и аудио: Мировые хиты]
4,[Спорт: Спортивное оборудование]


In [ ]:
def IoU(st_real, st_pred):
    st_real = set(st_real.split(", "))
    st_pred = set(st_pred.split(", "))

    up_value = st_real & st_pred
    down_value = st_real | st_pred

    return len(up_value) / len(down_value)

In [ ]:
res = 0
k_mines = 0
for i in range(data.shape[0]):
    real = data.iloc[i, 3]
    pred = sample_submission.iloc[i, 1][0]
    try:
      out = IoU(real, pred)
    except:
      print(real)
      print(pred)
      print("---")
      k_mines += 1
      out = 0
    res += out

print(res / (data.shape[0] - k_mines))

nan
Семья и отношения: Смерть родственников
---
0.020833333333333332


In [ ]:
0.012722646310432571

0.012722646310432571

In [ ]:
sample_submission.to_csv("sample_submission.csv", index_label=0)

# Second_part

Посмотрим на tag 1 уровня


In [223]:
train_data_cat.head()

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира


In [224]:
print(train_data_cat.shape)
template = [(isinstance(i, str)) for i in  train_data_cat["tags"].tolist()]
train_data_cat = train_data_cat[template]
print(train_data_cat.shape)

(1048, 4)
(1048, 4)


In [225]:
title_discription = []
tag_list = []

for i in train_data_cat.iterrows():
  tags = i[1].tags.split(", ")
  title = i[1].title
  description = i[1].description
  for j in range(len(tags)):
    tags[j] = tags[j].split(":")[0].strip()
    tags[j] = tags[j].split("\t")[0].strip()
    title_discription.append(title+". "+description)
    tag_list.append(tags[j])

df_work = pd.DataFrame({"title_discription": title_discription, "tag_list": tag_list})
df_work.head()

,title_discription,tag_list
0,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Массовая культура
1,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...",События и достопримечательности
2,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...",Массовая культура
3,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Массовая культура
4,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Массовая культура


In [226]:
df_work.tag_list.value_counts()

tag_list
Массовая культура                   559
События и достопримечательности     131
Спорт                               109
Карьера                             104
Религия и духовность                 78
Еда и напитки                        76
Путешествия                          74
Музыка и аудио                       66
Хобби и интересы                     59
Транспорт                            58
Семья и отношения                    50
Фильмы и анимация                    34
Дом и сад                            28
Наука                                23
Личные финансы                       19
Изобразительное искусство            18
Стиль и красота                      17
Образование                          16
Новости и политика                   11
Бизнес и финансы                     10
Игры                                  9
Информационные технологии             6
Книги и литература                    4
Недвижимость                          3
Медицинские направления        

In [128]:
all_tags = []
for i in IAB_tags.iterrows():
  a, b, c = i[1]["Уровень 1 (iab)"], i[1]["Уровень 2 (iab)"], i[1]["Уровень 3 (iab)"]
  if not isinstance(b, str):
    all_tags.append(a)
  else:
    if not isinstance(c, str):
      all_tags.append(a+": "+b)
    else:
      all_tags.append(a+": "+b+": "+c)

### Разделим tag поотдельности

In [129]:
title_discription = []
tag_list = []
tag_list_first = []

for i in train_data_cat.iterrows():
  tags = i[1].tags.split(", ")
  title = i[1].title
  description = i[1].description
  for j in range(len(tags)):
    tag_list.append(tags[j])
    tags[j] = tags[j].split(":")[0].strip()
    tags[j] = tags[j].split("\t")[0].strip()
    title_discription.append(title+". "+description)
    tag_list_first.append(tags[j])

df_work = pd.DataFrame({"title_discription": title_discription, "tag_list": tag_list, "tag_list_first": tag_list_first})
df_work.head()

,title_discription,tag_list,tag_list_first
0,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Массовая культура: Юмор и сатира,Массовая культура
1,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...",События и достопримечательности: Спортивные с...,События и достопримечательности
2,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...",Массовая культура: Юмор и сатира,Массовая культура
3,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Массовая культура: Отношения знаменитостей,Массовая культура
4,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Массовая культура: Скандалы знаменитостей,Массовая культура


Какие есть выбросы

In [130]:
uniq_tags = taxonomy["Уровень 1 (iab)"].unique().tolist()
# for i in uniq_tags:
#     dop_tags = taxonomy[taxonomy["Уровень 1 (iab)"] == i]["Уровень 2 (iab)"].unique().tolist()
#     uniq_tags += dop_tags
ttttags = df_work.tag_list_first.unique().tolist()

for i in ttttags:
  if i not in uniq_tags:
    print(i)


Медицинские направления
Экономика
Игры и головоломки
Компьютеры и цифровые технологии
Красота
Хобби и стиль
Отношения знаменитостей
Создание контента


In [131]:
# -> all_tags
# for i in uniq_tags:
#     dop_tags = taxonomy[taxonomy["Уровень 1 (iab)"] == i]["Уровень 2 (iab)"].unique().tolist()
#     uniq_tags += dop_tags
ttttags = df_work.tag_list.unique().tolist()

k = 0
for i in ttttags:
  if i not in all_tags:
    print(i)
    k += 1
print(k)

События и достопримечательности:  Спортивные события
Религия и духовность: астрология

Медицинские направления: Психиатрия
Фильмы и анимация: Фильмы и анимация 
Семья и отношения: Воспитание детей% Воспитание малышей
Наука	Космос и астрономия
Образование	Онлайн-образование
События и достопримечательности	Концерты и музыкальные мероприятия
Экономика: Валюты
Игры и головоломки: Карточные игры
Компьютеры и цифровые технологии
Красота: Макияж
Хобби и стиль
Отношения знаменитостей: Семьи знаменитостей
Спорт:Борьба
Создание контента: Личные события
Спорт:Футбол
События и достопримечательности:Личные события

Путешествия: Путешествия на машине


Массовая культура: Стиль знаменитостей: Стиль знаменитостей

Музыка и аудио: Современная музыка
Массовая культура:Отношения знаменитостей
Транспорт:Авторемонт
Религия и духовность:Астрология
Транспорт:Типы автомобилей:Классические автомобили

Массовая культура: Отношения знаменитостей
Музыка и аудио:Современная музыка
Изобразительное искусство:Художес

In [132]:
print(df_work.shape)
df_work = df_work[[i in all_tags for i in df_work.tag_list.tolist()]]
print(df_work.shape)

(1578, 3)
(1458, 3)


Сколько всего sample у каждого 1 уровня

In [133]:
uniq = set(df_work.tag_list_first.tolist())
r_list = df_work.tag_list_first.tolist()
for i in uniq:
  print(i, r_list.count(i))

Еда и напитки 75
Недвижимость 3
Путешествия 73
Хобби и интересы 59
Личные финансы 19
Стиль и красота 17
Спорт 105
Книги и литература 4
Игры 9
События и достопримечательности 128
Карьера 103
Транспорт 56
Семья и отношения 49
Дом и сад 28
Музыка и аудио 62
Образование 15
Информационные технологии 6
Здоровый образ жизни 1
Религия и духовность 6
Массовая культура 555
Новости и политика 11
Бизнес и финансы 9
Наука 22
Телевидение 2
Фильмы и анимация 23
Изобразительное искусство 16
Медицина 2


In [134]:
df_work.head()

,title_discription,tag_list,tag_list_first
0,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Массовая культура: Юмор и сатира,Массовая культура
2,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...",Массовая культура: Юмор и сатира,Массовая культура
3,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Массовая культура: Отношения знаменитостей,Массовая культура
4,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Массовая культура: Скандалы знаменитостей,Массовая культура
5,$1000 шоу | 1 выпуск | Автобоулинг. В этом вып...,Транспорт,Транспорт


Для стратификации нужно добавить 1 элемент

In [135]:
new_row = {'title_discription': 'Здоровый образ жизни', 'tag_list': 'Здоровый образ жизни', 'tag_list_first': 'Здоровый образ жизни'}
new_row_df = pd.DataFrame([new_row])
df_work = pd.concat([df_work, new_row_df], ignore_index=True)

### Разделяем на train.test

In [136]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_work, test_size=0.3, stratify=df_work['tag_list_first'], random_state=42)

print("Train size:", len(train_df))
print("Test size:", len(test_df))

print("Train class distribution:")
print(train_df['tag_list_first'].value_counts())

print("\nTest class distribution:")
print(test_df['tag_list_first'].value_counts())

Train size: 1021
Test size: 438
Train class distribution:
tag_list_first
Массовая культура                  388
События и достопримечательности     90
Спорт                               74
Карьера                             72
Еда и напитки                       53
Путешествия                         51
Музыка и аудио                      43
Хобби и интересы                    41
Транспорт                           39
Семья и отношения                   34
Дом и сад                           20
Фильмы и анимация                   16
Наука                               15
Личные финансы                      13
Стиль и красота                     12
Изобразительное искусство           11
Образование                         11
Новости и политика                   8
Игры                                 6
Бизнес и финансы                     6
Религия и духовность                 4
Информационные технологии            4
Книги и литература                   3
Недвижимость                  

### Обучение triplet loss


In [137]:
import torch

torch.cuda.empty_cache()

In [195]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import random
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = SentenceTransformer('intfloat/multilingual-e5-small')
model = model.to('cuda')

In [196]:
from tqdm import tqdm

In [197]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)

class TripletDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.classes = dataframe['tag_list_first'].unique()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        anchor_class = self.data.iloc[idx]['tag_list_first']

        positive = self.data.iloc[idx]['title_discription']

        negative_samples = self.data[self.data['tag_list_first'] != anchor_class]
        negative = negative_samples.sample(1).iloc[0]['title_discription']

        return anchor_class, positive, negative

train_dataset = TripletDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


def tokenize_batch(texts, device):
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    return {key: value.to(device) for key, value in tokens.items()}

def train_triplet_loss_model(model, train_loader, optimizer, device, epochs=2):
    model.train()
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for batch in tqdm(train_loader):
            anchors, positives, negatives = batch

            # Токенизация текстов и перенос на устройство
            anchor_tokens = tokenize_batch(anchors, device)
            positive_tokens = tokenize_batch(positives, device)
            negative_tokens = tokenize_batch(negatives, device)

            # Прямой проход через модель для получения эмбеддингов
            anchor_embeddings = model(anchor_tokens)['sentence_embedding']
            positive_embeddings = model(positive_tokens)['sentence_embedding']
            negative_embeddings = model(negative_tokens)['sentence_embedding']

            # Вычисление потерь
            loss = triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)

            # Обратное распространение ошибки
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

# Запуск обучения с указанием устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_triplet_loss_model(model, train_loader, optimizer, device, epochs=2)

 50%|█████     | 1/2 [00:07<00:07,  7.67s/it]

Epoch 1, Loss: 0.8393236319534481



100%|██████████| 2/2 [00:14<00:00,  7.48s/it]

Epoch 2, Loss: 0.48967784363776445


In [198]:
tags_list = list(set(df_work.tag_list_first.unique().tolist()))
vectors = np.array([model.encode(i) for i in tags_list])

dim = vectors.shape[1]  # Размерность векторов
index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
index.add(vectors)

In [199]:
all_pred_1 = []
all_pred_2 = []
all_pred_3 = []

k = 0
for i, row in test_df.iterrows():
    real = row['tag_list_first']

    if not isinstance(row['title_discription'], str):
      all_pred_1.append("-")
      all_pred_2.append("-")
      all_pred_3.append("-")
      continue
    query_vector = model.encode(row['title_discription'])
    query_vector = np.array(query_vector).astype('float32')
    D, predictions = index.search(np.expand_dims(query_vector, axis=0), 3)

    pred_1, pred_2, pred_3 = tags_list[predictions[0][0]], tags_list[predictions[0][1]], tags_list[predictions[0][2]]
    all_pred_1.append(pred_1)
    all_pred_2.append(pred_2)
    all_pred_3.append(pred_3)
    if real in [pred_1, pred_2, pred_3]:
      k += 1

In [200]:
print(k, "/", len(test_df))

302 / 438


In [203]:
df_dop = pd.DataFrame({"q1": all_pred_1, "q2": all_pred_2, "q3": all_pred_3})
test_df_new = pd.concat([test_df.reset_index(drop=True), df_dop.reset_index(drop=True)], axis=1)
test_df_new.head()

,title_discription,tag_list,tag_list_first,q1,q2,q3
0,«МузЛофт - Подкаст» с Ириной Нельсон - 1 июля ...,Музыка и аудио,Музыка и аудио,Музыка и аудио,Карьера,Изобразительное искусство
1,Музлофт-Подкаст с Рыбиным - 14 августа в 15:00...,События и достопримечательности: Концерты и му...,События и достопримечательности,Музыка и аудио,Карьера,Изобразительное искусство
2,"«МузЛофт - Подкаст» с Мартиросяном, Сорокиным,...",Музыка и аудио: Комедия и стендап (Музыка и ау...,Музыка и аудио,Музыка и аудио,Карьера,Изобразительное искусство
3,Музлофт-Подкаст | Виктор Салтыков. Конфликты с...,Массовая культура,Массовая культура,Музыка и аудио,Карьера,Телевидение
4,Команда 3/21 в ГрандТуре «Байкальская миля 202...,События и достопримечательности: Исторические ...,События и достопримечательности,Путешествия,События и достопримечательности,Транспорт


In [204]:
test_df_new.shape

(438, 6)

# Обучение сиамской модели

In [205]:
model_small.encode("123").shape

(384,)

In [206]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

df_work.tag_list.value_counts()

### Обучим на "Массовая культура"

In [207]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [208]:
class EmbeddingPairsDataset(Dataset):
    def __init__(self, embeddings1, embeddings2, labels):
        self.embeddings1 = embeddings1
        self.embeddings2 = embeddings2
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        emb1 = torch.tensor(self.embeddings1[idx], dtype=torch.float32)
        emb2 = torch.tensor(self.embeddings2[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return emb1, emb2, label

In [209]:
class SiameseNetwork_1(nn.Module):
    def __init__(self, embedding_dim=384):
        super(SiameseNetwork, self).__init__()

        self.head1 = nn.Sequential(
            nn.Linear(embedding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        self.head2 = nn.Sequential(
            nn.Linear(embedding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, emb1, emb2):
        output1 = self.head1(emb1)
        output2 = self.head2(emb2)

        diff = torch.abs(output1 - output2)

        output = self.classifier(diff)
        return output

In [210]:
train_df.head()

,title_discription,tag_list,tag_list_first
573,Артмеханика. Концерт рэпера ST.. Концерт рэпер...,События и достопримечательности: Концерты и му...,События и достопримечательности
1225,Артмеханика. Выпуск 13. Кондитерское искусство...,Массовая культура,Массовая культура
1022,Артмеханика. Интервью. Руслан Вяльцев.. Кто та...,Информационные технологии,Информационные технологии
769,«Давай сделаем это на кухне» с Викой Цыгановой...,Семья и отношения,Семья и отношения
785,Домашняя кухня от шефа: Выпуск №4. Мечтали сде...,Еда и напитки: Кулинария,Еда и напитки


In [217]:
# Подготовка данных
embeddings1 = torch.tensor([model.encode(i) for i in train_df.title_discription.tolist()], dtype=torch.float32)
embeddings2 = torch.tensor([model.encode("Массовая культура")] * embeddings1.shape[0], dtype=torch.float32)
labels = torch.tensor([1 if i == "Массовая культура" else 0 for i in train_df.tag_list_first.tolist()], dtype=torch.float32)

dataset = EmbeddingPairsDataset(embeddings1, embeddings2, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Инициализация модели, функции потерь и оптимизатора
model_Siame = SiameseNetwork_1(embedding_dim=384).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_Siame.parameters(), lr=0.001)

num_epochs = 25
model_Siame.train()

# Обучение модели
for epoch in range(num_epochs):
    epoch_loss = 0
    for emb1, emb2, label in dataloader:
        emb1 = emb1.to(device)
        emb2 = emb2.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        output = model_Siame(emb1, emb2).squeeze()

        loss = criterion(output, label)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}')

/tmp/ipykernel_921/4082777534.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb1 = torch.tensor(self.embeddings1[idx], dtype=torch.float32)
/tmp/ipykernel_921/4082777534.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb2 = torch.tensor(self.embeddings2[idx], dtype=torch.float32)
/tmp/ipykernel_921/4082777534.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 1/25, Loss: 0.5914238388650119
Epoch 2/25, Loss: 0.48638823442161083
Epoch 3/25, Loss: 0.47756234696134925
Epoch 4/25, Loss: 0.46490820753388107
Epoch 5/25, Loss: 0.4631820279173553
Epoch 6/25, Loss: 0.4513613353483379
Epoch 7/25, Loss: 0.45646956702694297
Epoch 8/25, Loss: 0.4487421032972634
Epoch 9/25, Loss: 0.4474172294139862
Epoch 10/25, Loss: 0.44477017573080957
Epoch 11/25, Loss: 0.4469791376031935
Epoch 12/25, Loss: 0.4453283669427037
Epoch 13/25, Loss: 0.43650709954090416
Epoch 14/25, Loss: 0.43737277132458985
Epoch 15/25, Loss: 0.4295145063661039
Epoch 16/25, Loss: 0.42853265162557364
Epoch 17/25, Loss: 0.43489855993539095
Epoch 18/25, Loss: 0.4277565199881792
Epoch 19/25, Loss: 0.42129310593008995
Epoch 20/25, Loss: 0.4220160080585629
Epoch 21/25, Loss: 0.4164398554712534
Epoch 22/25, Loss: 0.41819892078638077
Epoch 23/25, Loss: 0.41774088703095913
Epoch 24/25, Loss: 0.41224406799301505
Epoch 25/25, Loss: 0.410826968960464


In [221]:
# Сохранение модели
torch.save(model_Siame.state_dict(), 'siamese_model_1.pth')

In [218]:
model.train()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [219]:
# Подготовка тестовых данных
embeddings1_test = torch.tensor([model.encode(i) for i in test_df.title_discription.tolist()], dtype=torch.float32)
embeddings2_test = torch.tensor([model.encode("Массовая культура")] * embeddings1_test.shape[0], dtype=torch.float32)
labels_test = torch.tensor([1 if i == "Массовая культура" else 0 for i in test_df.tag_list_first.tolist()], dtype=torch.float32)

dataset_test = EmbeddingPairsDataset(embeddings1_test, embeddings2_test, labels_test)
dataloader_test = DataLoader(dataset_test, batch_size=16, shuffle=False)

In [220]:
# Оценка модели
res_all = 0
k = 0
model_Siame.eval()
with torch.no_grad():
    for emb1, emb2, label in dataloader_test:
        emb1 = emb1.to(device)
        emb2 = emb2.to(device)
        label = label.to(device)
        output = model_Siame(emb1, emb2).squeeze()
        prediction = (output > 0.5).float()
        res = torch.sum(torch.abs(label - output)) / len(label)
        res_all += res.item()
        k += 1

print(res_all / k)

0.3089067669851439


/tmp/ipykernel_921/4082777534.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb1 = torch.tensor(self.embeddings1[idx], dtype=torch.float32)
/tmp/ipykernel_921/4082777534.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb2 = torch.tensor(self.embeddings2[idx], dtype=torch.float32)
/tmp/ipykernel_921/4082777534.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], dtype=torch.float32)


### Обучение на События и достопримечательности

In [228]:
# Определяем устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [229]:
class EmbeddingPairsDataset(Dataset):
    def __init__(self, embeddings1, embeddings2, labels):
        self.embeddings1 = embeddings1
        self.embeddings2 = embeddings2
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        emb1 = torch.tensor(self.embeddings1[idx], dtype=torch.float32)
        emb2 = torch.tensor(self.embeddings2[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return emb1, emb2, label

In [230]:
class SiameseNetwork_1(nn.Module):
    def __init__(self, embedding_dim=384):
        super(SiameseNetwork, self).__init__()

        self.head1 = nn.Sequential(
            nn.Linear(embedding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        self.head2 = nn.Sequential(
            nn.Linear(embedding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, emb1, emb2):
        output1 = self.head1(emb1)
        output2 = self.head2(emb2)

        diff = torch.abs(output1 - output2)

        output = self.classifier(diff)
        return output

In [236]:
# Подготовка данных
embeddings1 = torch.tensor([model.encode(i) for i in train_df.title_discription.tolist()], dtype=torch.float32)
embeddings2 = torch.tensor([model.encode("События и достопримечательности")] * embeddings1.shape[0], dtype=torch.float32)
labels = torch.tensor([1 if i == "События и достопримечательности" else 0 for i in train_df.tag_list_first.tolist()], dtype=torch.float32)

dataset = EmbeddingPairsDataset(embeddings1, embeddings2, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Инициализация модели, функции потерь и оптимизатора
model_Siame = SiameseNetwork_1(embedding_dim=384).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_Siame.parameters(), lr=0.001)

num_epochs = 25
model_Siame.train()

# Обучение модели
for epoch in range(num_epochs):
    epoch_loss = 0
    for emb1, emb2, label in dataloader:
        emb1 = emb1.to(device)
        emb2 = emb2.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        output = model_Siame(emb1, emb2).squeeze()

        loss = criterion(output, label)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}')

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
# Сохранение модели
torch.save(model_Siame.state_dict(), 'siamese_model_1.pth')

In [ ]:
# Подготовка тестовых данных
embeddings1_test = torch.tensor([model.encode(i) for i in test_df.title_discription.tolist()], dtype=torch.float32)
embeddings2_test = torch.tensor([model.encode("События и достопримечательности")] * embeddings1_test.shape[0], dtype=torch.float32)
labels_test = torch.tensor([1 if i == "События и достопримечательности" else 0 for i in test_df.tag_list_first.tolist()], dtype=torch.float32)

dataset_test = EmbeddingPairsDataset(embeddings1_test, embeddings2_test, labels_test)
dataloader_test = DataLoader(dataset_test, batch_size=16, shuffle=False)

In [ ]:
# Оценка модели
res_all = 0
k = 0
model_Siame.eval()
with torch.no_grad():
    for emb1, emb2, label in dataloader_test:
        emb1 = emb1.to(device)
        emb2 = emb2.to(device)
        label = label.to(device)
        output = model_Siame(emb1, emb2).squeeze()
        prediction = (output > 0.5).float()
        res = torch.sum(torch.abs(label - output)) / len(label)
        res_all += res.item()
        k += 1

print(res_all / k)

In [232]:
embeddings1 = np.array([model.encode(i) for i in train_df.title_discription.tolist()])

In [233]:
embeddings2 = np.array([model.encode("События и достопримечательности")]*embeddings1.shape[0])
labels = np.array([1 if i == "События и достопримечательности" else 0 for i in train_df.tag_list_first.tolist()])

In [234]:
dataset = EmbeddingPairsDataset(embeddings1, embeddings2, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

model_Siame = SiameseNetwork(embedding_dim=768)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_Siame.parameters(), lr=0.001)

num_epochs = 50
model_Siame.train()

for epoch in range(num_epochs):
    epoch_loss = 0
    for emb1, emb2, label in dataloader:
        optimizer.zero_grad()

        output = model_Siame(emb1, emb2).squeeze()


        loss = criterion(output, label)
        epoch_loss += loss.item()


        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x384 and 768x128)

In [ ]:
embeddings1_test = np.array([model.encode(i) for i in test_df.title_discription.tolist()])
embeddings2_test = np.array([model.encode("События и достопримечательности")]*embeddings1.shape[0])
labels_test = np.array([1 if i == "События и достопримечательности" else 0 for i in test_df.tag_list_first.tolist()])

In [ ]:
dataset_test = EmbeddingPairsDataset(embeddings1_test, embeddings2_test, labels_test)
dataloader_test = DataLoader(dataset, batch_size=16, shuffle=True)

In [193]:
res_all = 0
k = 0
model_Siame.eval()
with torch.no_grad():
    for emb1, emb2, label in dataloader_test:
        output = model_Siame(emb1, emb2).squeeze()
        prediction = (output > 0.5).float()
        res = sum(abs(label - output)) / len(label)
        res_all += res
        k += 1

print(res_all / k)

tensor(0.2518)
